# Transformación con centro de escalado y factores por eje

## Idea general

Si tienes un centro de escalado $ C = (C_x, C_y, C_z) $ y factores distintos por eje $ S_x, S_y, S_z $,  
la transformación que aplica ese escalado alrededor de $ C $ a un punto $ P = (x, y, z) $ es:

$$ 
P' = C + \big(S_x (x - C_x),\; S_y (y - C_y),\; S_z (z - C_z)\big)
 $$

En forma matricial (afín, con coordenadas homogéneas):

$$ 
M =
\begin{pmatrix}
S_x & 0   & 0   & C_x - S_x C_x \\
0   & S_y & 0   & C_y - S_y C_y \\
0   & 0   & S_z & C_z - S_z C_z \\
0   & 0   & 0   & 1
\end{pmatrix}
 $$

Aplicación:  
$$ 
[x', y', z', 1]^T = M [x, y, z, 1]^T
 $$

---

## Cómo obtener $ S_x, S_y, S_z $ a partir de correspondencias $ P_i \rightarrow Q_i $

Para cada par de puntos correspondientes:

$$
\mathbf{v}_i = P_i - C, \quad \mathbf{w}_i = Q_i - C
$$

Si el escalado es por ejes, se cumple componente a componente:

$$
w_{i,x} = S_x v_{i,x}, \quad w_{i,y} = S_y v_{i,y}, \quad w_{i,z} = S_z v_{i,z}
$$

### Cálculo directo

Cuando $ v_{i,axis} \neq 0 $:

$$
S_{axis} = \frac{w_{i,axis}}{v_{i,axis}}
$$

### Cálculo robusto (mínimos cuadrados)

Si tienes varias parejas de puntos:

$$
S_x = \frac{\sum_i v_{i,x}\,w_{i,x}}{\sum_i v_{i,x}^2}, \quad
S_y = \frac{\sum_i v_{i,y}\,w_{i,y}}{\sum_i v_{i,y}^2}, \quad
S_z = \frac{\sum_i v_{i,z}\,w_{i,z}}{\sum_i v_{i,z}^2}
$$

> ⚠️ Si algún denominador es 0, significa que no hay información sobre ese eje.

---

## Ejemplo con nodos

### Datos

Triángulo original:
- $ P_1 = (10, 0, 0) $
- $ P_2 = (10, 2, 0) $
- $ P_3 = (12, 2, 0) $

Triángulo escalado:
- $ Q_1 = (10, 0, 0) $
- $ Q_2 = (10, 1, 0) $
- $ Q_3 = (11, 1, 0) $

Centro de escalado:
$$
C = (10, 0, 0)
$$

### Cálculo de vectores

$$
\mathbf{v}_1 = (0, 0, 0), \quad \mathbf{w}_1 = (0, 0, 0)
$$
$$
\mathbf{v}_2 = (0, 2, 0), \quad \mathbf{w}_2 = (0, 1, 0)
$$
$$
\mathbf{v}_3 = (2, 2, 0), \quad \mathbf{w}_3 = (1, 1, 0)
$$

### Factores de escala

- Eje X: $ S_x = 1/2 = 0.5 $
- Eje Y: $ S_y = 1/2 = 0.5 $
- Eje Z: sin información → $ S_z = 0.5 $ (o $ 1 $ si se desea mantener Z)

### Matriz afín resultante (con $ S_z = 0.5$ )

$$
M =
\begin{pmatrix}
0.5 & 0 & 0 & 5 \\
0 & 0.5 & 0 & 0 \\
0 & 0 & 0.5 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}
$$

Comprobación con $ P_3 = (12, 2, 0) $ :

$$
P_3' = C + (0.5 \cdot (12-10),\; 0.5 \cdot (2-0),\; 0.5 \cdot (0-0)) = (11, 1, 0)
$$

Coincide con $ Q_3 $.

---

## Resumen práctico

1. Elegir/obtener el centro de escalado $ C $.  
2. Calcular los vectores $ \mathbf{v}_i = P_i - C $ y $ \mathbf{w}_i = Q_i - C $ .  
3. Calcular los factores $ S_x, S_y, S_z $ con la fórmula de mínimos cuadrados.  
4. Construir la matriz afín $ M $ y aplicarla a los puntos.

---

In [ ]:
## Implementación en Python

import numpy as np

##############################################
##############################################

# Ejemplo con tus nodos
P = [(10,0,0), (10,2,0), (12,2,0)]
Q = [(10,0,0), (10,1,0), (11,1,0)]
C = (10,0,0)

##############################################
##############################################

def scale_factors(P, Q, C):
    """
    Calcula los factores de escala por eje y la matriz afín 4x4.
    P, Q: listas (o arrays) Nx3 con coordenadas originales y transformadas.
    C: centro de escalado (3,)
    """
    P, Q, C = np.array(P), np.array(Q), np.array(C)
    v = P - C
    w = Q - C

    # Mínimos cuadrados por eje
    S = []
    for axis in range(3):
        num = np.sum(v[:, axis] * w[:, axis])
        den = np.sum(v[:, axis] ** 2)
        S.append(num / den if den != 0 else 1.0)
    Sx, Sy, Sz = S

    # Matriz afín
    M = np.array([
        [Sx, 0,  0,  C[0] - Sx * C[0]],
        [0,  Sy, 0,  C[1] - Sy * C[1]],
        [0,  0,  Sz, C[2] - Sz * C[2]],
        [0,  0,  0,  1]
    ])
    return Sx, Sy, Sz, M

Sx, Sy, Sz, M = scale_factors(P, Q, C)
print(f"Sx={Sx:.3f}, Sy={Sy:.3f}, Sz={Sz:.3f}")
print("Matriz afín:\n", M)

Sx=0.500, Sy=0.500, Sz=1.000
Matriz afín:
 [[0.5 0.  0.  5. ]
 [0.  0.5 0.  0. ]
 [0.  0.  1.  0. ]
 [0.  0.  0.  1. ]]
